#  Music Playlist Recommendation Engine

--------------------

## Problem Statement

In 2019, Spotify reached a major milestone of having more than 100MM paid [Spotify Premium users](https://www.engadget.com/2019/04/29/spotify-has-100-million-premium-users/). In fact, the number of paid subscribers grew by an impressive 32% YoY. Including free subscribers, Spotify now has a total of 217 million monthly active users worldwide. Key to this growth has been Spotify's curated playlists and personalized music recommendations. Personally, I've found that Spotify's recommendations via the Discover Weekly playlist have been essential to shaping my music taste over the past 2 years. As my preferences have evolved, so have my recommendations, as they've both adpated to my new direction while still allowing for discovery of new genres.

The goal of this analysis is to leverage publicly available Spotify song data to build a recommendation engine that could be used to extend a user playlist or a Spotify curated playlist. While Spoitfy makes an array of track/artist meta data available via their API, this recommendation engine will focus on leveraging artist meta data, associated artist genres, track meta data, and the audio features of a song.

While the immediate business use-case is specific music recommendations, the broader application of this analysis could be applied to any item-based recommendation engine (i.e. product recommendations on an e-commerce site).

As a secondary deliverable, I'd like to develop a web application within Slack that would take a user_id, playlist, discover/popular preference, and provide a list of recommended songs.

---------------

## Data Collection

The data for this analysis was collected via the [Spotipy API](https://spotipy.readthedocs.io/en/latest/). To generate the song library for our recommendation engine, we imported songs from +700 spotify curated playlists spanning all genres (i.e. Hip-Hop, Pop, Dance, Country, Jazz, etc).

The data collected included track meta data (i.e. track popularity, duration), data asociated with the artist (i.e. genres, popularity, follower count), and detailed audio analysis of each track (i.e. danceability, acousticness, tempo, etc.)

---------------------

## Exploratory Data Analysis

The dataset contained ~30K songs that spanned 21 genre categories including hip-hop, edm-dance, pop, reggae, and rock. While Spotify provides song data in most of the countries it operates in, this analysis is limited to playlists/songs available in the US only.

---------------------------------

**Top Artists in the Song Library:**

![image](./images/top_artists.png)

As expected, we can see that the top artists are big names like Lady Gaga, Kanye West, and Drake. We'll plot this to see it visually

**Top Genres in the Song Library:**

![image](./images/top_genres.png)

**Distribution of Genres per Artist:**

![image](./images/genres_per_artist.png)

We can see the majority of artist have one genre associated with them, but the distribution is certainly right skewed with a long tail

**Distribution of Track Acousticness:**

![image](./images/acousticness.png)

There are a number of track features that measure the probability/confidence that a track has a given attribute. For example, `acousticness` is the predicted probability that a track is compromised entirely of acoustic sound. A zero would indicate the track has a 0.0% probability of being entirely acoustic, thus we are 100% confident that it contains non-acoustic music.

**Distribution of Track Danceability:**

![image](./images/danceability.png)

**Correlation Heatmap:**

![image](./images/heatmap.png)

**Key Takeaways:**

- First we see that none of the track features are strongly correlated with `artist_popularity` and `track_popularity` which tells us intuitively that popular tracks/ artists in our library represent a diverse collection of music

- Features with the highest postive correlation are `loudness` & `energy` (0.75).

- `Valence` and `danceability` also have a moderately positive correlation at 0.49, indicating that songs that are more upbeat are more danceable (makes sense)

- As expected, `track_popularity` and `artist_popularity` have a strong positive correlation of 0.74

- We also see that `acousticness` and features like `energy`/`loudness` have a strong negative correlation

---------

## Recommendation System Overview

### Item-Based Collaborative Filtering

Item-based collaborative filtering is a model-based algorithm for making recommendations. In the algorithm, the similarities between different items in the dataset are calculated by using one of a number of similarity measures (Cosine Similarity in this case).

--------

## Modeling Concepts

### Singular Value Decomposition (SVD)

Singular Value Decomposition (SVD) is a dimensionality reduction technique in linear algebra that factorizes any matrix M into the product of 3 separate matrices. The ultimate goal being to reduce a matrix to its constituent parts in order to make certain subsequent matrix calculations simpler.

The technique is widely used in the context of Natural Language Processing (NLP) as well. Topic modeling is the process of discovering groups of co-occurring words in text documents. These groups co-occurring words makes are related in some way, and can be considered 'topics'.

In this analysis, we'll use SVD to both reduce the dimensionalty of our track dataframe and as a means of analyzing commonly co-occurring artist genres to infor future recommendations.

### HDBSCAN

Hierarchical Density-Based Spatial Clustering of Applications with Noise (HDBSCAN) is a clustering algorithm developed by Campello, Moulavi, and Sander. It extends DBSCAN by converting it into a hierarchical clustering algorithm, and then using a technique to extract a flat clustering based in the stability of clusters.

Notes:
- We can detect areas of high and low density

- Areas of high density will become a cluster

- Areas of low density will be not clustered/regarded as noise



### Cosine Similarity

The below code will return a square matrix, comparing every track with every other track in the dataset and calculating a similarity measure between each

-------------

## Summary Findings

- Using matrix decomposition, specifically Singular Value Decomposition, we were able to correctly identify consistently co-occuring music genres in the song library. See previous notebook. These findings could be leveraged to genre based predictions in the future.

- Analysis of the track recommendations based on Cosine Similarity first prioritized other tracks by that artist, then resulted in a mix of tracks consistent with the artist musical aesthetic. Spot checking a few artist our recommendations seem to make a lot of sense. Example for a Kanye West track below:

![image](./images/table.png)

- Clustering using HDBSCAN seems to provide some promising results. When assessing the playlist make-up of the largest clusters we certainly see that the tracks have a consistent themes such as country, peaceful songs, and chill/lo-fi music. In fact, the cluster algoirthm may benefit from a reduced number of clusters as there are currently 1.6k unique clusters

- Other Recommendation Considerations:

    - Recommendations based on most played tracks or most popular can serve as a good baseline to gauge the performance of other more complex recommender systems
    
    - Metrics to evaluate performance will defer depending on the business application. In the context of music recommendations, evaluation metrics might include avg. listening time of recommended songs. In the context of an e-commerce site, evaluation metric might be avg. incremental transactions. Evaluation metrics should identified prior to building the recommender
    
    - Balancing the ability to recommmend tracks we're confident the user will enjoy based on previous listening habits with the discovery of new music that may be outside their listeing habits is key. Introducing some randomness can lead to more discovery and greater engagement in the long run

-----------

## Limitations/Assumptions

- The genres in the meta data are in fact tied to the artist themselves, not the track itself. There may be cases where the artist creates some content that spans genres, or is vastly different than their typical style of song. In these cases, it's unclear whether the track features themselves or the artist genres will dominant the recommendations

------------

## Next Steps

- More work is needed to quantify the quality of the recommendations

- Transfer Learning of Clusters: As a next step we'lll want to incorporate the clusters generated from the HDBSCAN as a feature in our track data set. This may improve future recommedations

- Item-User Based Recommendations: While we don't have access to data within a user's library, representing playlists as users and buliding an item-user based collaborative filtering recommender would likely next step